# 동적 토픽 모델링(Dynamic Topic Modeling)
- 앞에서 수행한 토픽 트렌드 분석은 토픽들이 고정되어 있었음
- 시간이 지나면서 토픽 역시 변하기 때문에 이를 이용한 트렌드 분석이 필요
<br/>

- **동적 토픽 모델링**: 이전 토픽을 반영해 다음 시간의 토픽을 추출하는 것 <br/>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; 즉, **어떤 시점 t에서의 α, β가 이전 시점인 t-1에서의 α, β에 의해 결정된다고 가정** <br/><br/><br/><br/>


## 동적 토픽 모델링 원리
<img src="https://lyusungwon.github.io/assets/images/dynamic_topic_model.png" width="500" height="500"/>

그림의 각 열은 LDA 구조, **어떤 시점 t에서의 α, β가 이전 시점인 t-1에서의 α, β에 의해 결정된다고 가정** <br/> <br/>

<수식> αt | αt−1 ∼ N (αt−1,δ2I), &nbsp; &nbsp; βt | βt−1 ∼ N (βt−1,σ2I) <br/>
-> αt와 βt는 이전 시점의 αt−1, βt−1를 평균으로 하는 정규분포로부터 샘플링됨을 의미 <br/> <br/> <br/>

In [1]:
import pandas as pd

df = pd.read_csv("../data/petition_sampled.csv")
df_sorted = df.sort_values(by='start')     # 청원 날짜 순으로 정렬

# time slice 생성
df_sorted['time'] = df_sorted['start'].map(lambda x: x[:7])      # 월까지만 추출해서 time 컬럼 생성
print(df_sorted['time'][:3]) 

# 월별 문서 개수를 time_slice에 저장
time_slice = list(df_sorted['time'].value_counts().sort_index())
print(time_slice, sum(time_slice))

0    2017-08
1    2017-08
2    2017-08
Name: time, dtype: object
[83, 906, 298, 1446, 926, 1514, 1304, 1097, 1217, 1301, 1246, 1291, 1390, 1155, 1277, 1187, 439] 18077


In [4]:
# 시간 구간을 리스트로 작성
time_tag = sorted(list(set(df_sorted['time'])))

# 제대로 대응되는지 확인
print(time_tag, time_slice)
print(df_sorted['time'].value_counts().sort_index())

['2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12'] [83, 906, 298, 1446, 926, 1514, 1304, 1097, 1217, 1301, 1246, 1291, 1390, 1155, 1277, 1187, 439]
2017-08      83
2017-09     906
2017-10     298
2017-11    1446
2017-12     926
2018-01    1514
2018-02    1304
2018-03    1097
2018-04    1217
2018-05    1301
2018-06    1246
2018-07    1291
2018-08    1390
2018-09    1155
2018-10    1277
2018-11    1187
2018-12     439
Name: time, dtype: int64


In [5]:
# dictionary, corpus 생성
from gensim.corpora.dictionary import Dictionary
from konlpy.tag import Okt

t = Okt()

def tokenizer(doc): #명사만 사용
    return [token for token in t.nouns(doc) if len(token) > 1]

#청원 텍스트를 토큰화
texts = [tokenizer(news) for news in df_sorted['content']]

# 토큰화 결과로부터 dictionay 생성
dictionary = Dictionary(texts) 
print('#Number of initial unique words in documents:', len(dictionary))

# 문서 빈도수가 너무 적거나 높은 단어를 필터링하고 특성을 단어의 빈도 순으로 선택
dictionary.filter_extremes(keep_n = 2000, no_below=5, no_above=0.5)
print('#Number of unique words after removing rare and common words:', len(dictionary))

# 카운트 벡터로 변환 -> corpus 생성
corpus = [dictionary.doc2bow(text) for text in texts]
print('#Number of unique tokens: %d' % len(dictionary))
print('#Number of documents: %d' % len(corpus))

#Number of initial unique words in documents: 36344
#Number of unique words after removing rare and common words: 2000
#Number of unique tokens: 2000
#Number of documents: 18077


In [ ]:
# LdaSeqModel